In [5]:
#generate pb for ESPCN
import os, time
import tensorflow as tf
import imageio
from subpixel import *
from utils import *

%reload_ext autoreload
%autoreload 2

W = 720 #720
H = 480 #480
scale = 3
mode = "YCbCr"#"RGB"
channel = 3
xx = True
if mode == "YCbCr":
    channel = 1
    xx = False
model_name = "ESPCN-"+str(mode)+"-x"+str(scale)
pb_name = "ESPCN-"+str(mode)+"-"+str(H)+"-x"+str(scale)+".pb"

W1 = 0
W2 = 0
W3 = 0
W4 = 0
B1 = 0
B2 = 0
B3 = 0
B4 = 0

tf.reset_default_graph()

sess = tf.Session()
with sess.as_default():
    input_LR = tf.placeholder(tf.float32, [None, H, W, channel], name='input')
    feature_tmp = tf.layers.conv2d(input_LR, 64, 5, strides = 1, padding = 'SAME', name = 'CONV_1',
                                kernel_initializer = tf.contrib.layers.xavier_initializer(), reuse=tf.AUTO_REUSE)
    feature_tmp = tf.nn.relu(feature_tmp)
    feature_tmp = tf.layers.conv2d(feature_tmp, 32, 3, strides = 1, padding = 'SAME', name = 'CONV_2',
                                kernel_initializer = tf.contrib.layers.xavier_initializer(), reuse=tf.AUTO_REUSE)
    feature_tmp = tf.nn.relu(feature_tmp)
    feature_out = tf.layers.conv2d(feature_tmp, channel*scale*scale, 3, strides = 1, padding = 'SAME', 
                   name = 'CONV_3', kernel_initializer = tf.contrib.layers.xavier_initializer(), reuse=tf.AUTO_REUSE)
    feature_out = PS(feature_out, scale, color=xx)
    output_HR = tf.layers.conv2d(feature_out, channel, 1, strides = 1, padding = 'SAME', 
                        name = 'CONV_OUT', kernel_initializer = tf.contrib.layers.xavier_initializer(), reuse=tf.AUTO_REUSE)
    #output_HR = tf.add(tf.multiply(tf.add(tf.nn.tanh(feature_out),1),(235-16)/2), 16, name = 'output') #Y range is in 16-235
    init_op = tf.global_variables_initializer() 
    
    saver = tf.train.Saver()
    saver.restore(sess, os.path.join("./checkpoint/DIV2K", model_name))
    vv = [v for v in tf.global_variables()]
    print("variable list: ", vv)
    W1 = [v for v in tf.global_variables() if v.name == "CONV_1/kernel:0"][0].eval()
    W2 = [v for v in tf.global_variables() if v.name == "CONV_2/kernel:0"][0].eval()
    W3 = [v for v in tf.global_variables() if v.name == "CONV_3/kernel:0"][0].eval()
    W4 = [v for v in tf.global_variables() if v.name == "CONV_OUT/kernel:0"][0].eval()
    B1 = [v for v in tf.global_variables() if v.name == "CONV_1/bias:0"][0].eval()
    B2 = [v for v in tf.global_variables() if v.name == "CONV_2/bias:0"][0].eval()
    B3 = [v for v in tf.global_variables() if v.name == "CONV_3/bias:0"][0].eval()
    B4 = [v for v in tf.global_variables() if v.name == "CONV_OUT/bias:0"][0].eval()

tf.reset_default_graph()
sess2 = tf.Session()
print("graph reset!")
with sess2.as_default():
    input_LR = tf.placeholder(tf.float32, [1, H, W, channel], name='input')
    feature_tmp = tf.nn.conv2d(input_LR, W1, strides=[1, 1, 1, 1], padding='SAME')
    feature_tmp = tf.nn.bias_add(feature_tmp, B1)
    feature_tmp = tf.nn.relu(feature_tmp)
    feature_tmp = tf.nn.conv2d(feature_tmp , W2, strides=[1, 1, 1, 1], padding='SAME')
    feature_tmp = tf.nn.bias_add(feature_tmp, B2)
    feature_tmp = tf.nn.relu(feature_tmp)
    feature_tmp = tf.nn.conv2d(feature_tmp , W3, strides=[1, 1, 1, 1], padding='SAME')
    feature_out = tf.nn.bias_add(feature_tmp, B3)
    feature_out = PS(feature_out, scale, color=xx)
    output_HR = tf.nn.conv2d(feature_out, W4, strides=[1,1,1,1], padding='SAME')
    output_HR = tf.nn.bias_add(output_HR, B4)
    output_HR = tf.clip_by_value(output_HR, 0, 255, name = 'output')
    #output_HR = tf.add(tf.multiply(tf.add(tf.nn.tanh(feature_out),1),(235-16)/2), 16, name = 'output') #Y range is in 16-235
    tf.train.write_graph(sess2.graph_def, "./checkpoint/pb", pb_name, as_text=False)
    #tf.train.write_graph(sess2.graph_def, "./checkpoint/pb", 'ESPCN.pbtxt', as_text=True)
    #LOGDIR='./logs'
    #train_writer = tf.summary.FileWriter(LOGDIR)
    #train_writer.add_graph(sess2.graph)
    vv = [v for v in tf.global_variables()]
    print("Drew new graph! variable list: ", vv)

INFO:tensorflow:Restoring parameters from ./checkpoint/DIV2K/ESPCN-YCbCr-x3
variable list:  [<tf.Variable 'CONV_1/kernel:0' shape=(5, 5, 1, 64) dtype=float32_ref>, <tf.Variable 'CONV_1/bias:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'CONV_2/kernel:0' shape=(3, 3, 64, 32) dtype=float32_ref>, <tf.Variable 'CONV_2/bias:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'CONV_3/kernel:0' shape=(3, 3, 32, 9) dtype=float32_ref>, <tf.Variable 'CONV_3/bias:0' shape=(9,) dtype=float32_ref>, <tf.Variable 'CONV_OUT/kernel:0' shape=(1, 1, 1, 1) dtype=float32_ref>, <tf.Variable 'CONV_OUT/bias:0' shape=(1,) dtype=float32_ref>]
graph reset!
Drew new graph! variable list:  []


In [7]:
img = load_image(os.path.join("/home/johnyi/deeplearning/research/SISR_Datasets/test/Set5/baby.png"), mode)
img = doresize(img, [H, W])
Y = np.split(img, 3, axis=2)[0]
Cb = np.split(img, 3, axis=2)[1]
Cr = np.split(img, 3, axis=2)[2]
start_time = time.time()
[out] = sess2.run([output_HR], feed_dict={input_LR: [Y]})
print("runtime: %.6f" %(time.time()-start_time))
#out = np.round(out).astype("uint8")
out = out.astype("uint8")
print("out shape:",out.shape)
save_ycbcr_img(out[0], Cb, Cr, 3, "test.png")
#imageio.imwrite("test.png",out[0])

runtime: 0.010543
out shape: (1, 1440, 2160, 1)


0

In [3]:
# Test loading pb file (YCbCr)
from tensorflow.core.framework import graph_pb2
import os
import time
import numpy as np

W = 256
H = 256
scale = 3
mode = "YCbCr"
channel = 3
xx = True
if mode == "YCbCr":
    channel = 1
    xx = False

pb_name = "ESPCN-"+str(mode)+"-"+str(H)+"-x"+str(scale)+".pb"
tf.reset_default_graph()
sess = tf.Session()
with sess.as_default():        
    with open(os.path.join('./checkpoint/pb/', pb_name), 'rb') as f:
        input_LR = tf.placeholder(tf.float32, [None, H, W, channel], name='input')
        #graph_def = tf.GraphDef()
        graph_def = graph_pb2.GraphDef()
        graph_def.ParseFromString(f.read())
        output = tf.import_graph_def(graph_def, input_map={'input:0': input_LR}, return_elements=['output:0'])
        output = output[0]
        #x = sess.graph.get_operations()
        #print("ops:",x)
        print(output)
        
        img = load_image(os.path.join("/home/johnyi/deeplearning/research/SISR_Datasets/test/Set5/baby.png"), "YCbCr")
        img = doresize(img, [H, W])
        Y_bicubic = np.split(imresize(img,[scale*H, scale*W], interp='bicubic'),3,axis=2)[0]
        Y_LR = np.split(img,3, axis=2)[0]
        Cb_LR = np.split(img,3, axis=2)[1]
        Cr_LR = np.split(img,3, axis=2)[2]
        start_time = time.time()
        [out] = sess.run([output], feed_dict={input_LR: [Y_LR]})
        print("runtime: %.6f" %(time.time()-start_time))
        out = out.astype("uint8")
        print("out size:",out[0].shape, np.min(out[0]), np.max(out[0]))
        print("out size:",Cb_LR[0].shape, np.min(Cb_LR[0]), np.max(Cb_LR[0]))
        print("out size:",Cr_LR[0].shape, np.min(Cr_LR[0]), np.max(Cr_LR[0]))
        save_ycbcr_img(out[0], Cb_LR, Cr_LR, scale, "./test.png")
        save_ycbcr_img(Y_bicubic, Cb_LR, Cr_LR, scale, "./test_bicubic.png")
        
runtimes = []
for i in range(0,10):
    img = load_image(os.path.join("/home/johnyi/deeplearning/research/SISR_Datasets/test/Set5/baby.png"), "YCbCr")
    img = doresize(img, [H, W])
    Y_bicubic = np.split(imresize(img,[scale*H, scale*W], interp='bicubic'),3,axis=2)[0]
    Y_LR = np.split(img,3, axis=2)[0]
    Cb_LR = np.split(img,3, axis=2)[1]
    Cr_LR = np.split(img,3, axis=2)[2]
    start_time = time.time()
    [out] = sess.run([output], feed_dict={input_LR: [Y_LR]})
    runtime = time.time()-start_time
    print("%.6f" %(runtime))
    runtimes.append(runtime)
    #print("runtime: %.6f" %(time.time()-start_time))
    out = out.astype("uint8")
    #print("out size:",out[0].shape, np.min(out[0]), np.max(out[0]))
    #print("out size:",Cb_LR[0].shape, np.min(Cb_LR[0]), np.max(Cb_LR[0]))
    #print("out size:",Cr_LR[0].shape, np.min(Cr_LR[0]), np.max(Cr_LR[0]))
print('avg: %.6f' %(np.sum(runtimes)/len(runtimes)))

Tensor("import/output:0", shape=(?, 768, 768, 1), dtype=float32)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


runtime: 0.631590
out size: (768, 768, 1) 0 255
out size: (256, 1) 88 129
out size: (256, 1) 125 168
0.002280
0.002200
0.002264
0.002256
0.002234
0.002265
0.002338
0.002237
0.002183
0.002322
avg: 0.002258


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:52: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [5]:
img = load_image(os.path.join("/home/johnyi/deeplearning/research/SISR_Datasets/test/Set5/baby.png"), "YCbCr")
img = doresize(img, [H, W])
Y_bicubic = np.split(imresize(img,[scale*H, scale*W], interp='bicubic'),3,axis=2)[0]
Y_LR = np.split(img,3, axis=2)[0]
Cb_LR = np.split(img,3, axis=2)[1]
Cr_LR = np.split(img,3, axis=2)[2]
start_time = time.time()
[out] = sess.run([output], feed_dict={input_LR: [Y_LR]})
print("runtime: %.6f" %(time.time()-start_time))
out = np.round(out).astype("uint8")
print("out size:",out[0].shape, np.min(out[0]), np.max(out[0]))
print("out size:",Cb_LR[0].shape, np.min(Cb_LR[0]), np.max(Cb_LR[0]))
print("out size:",Cr_LR[0].shape, np.min(Cr_LR[0]), np.max(Cr_LR[0]))
#save_ycbcr_img(out[0], Cb_LR, Cr_LR, scale, "./test.png")
#save_ycbcr_img(Y_bicubic, Cb_LR, Cr_LR, scale, "./test_bicubic.png")

runtime: 0.002226
out size: (768, 768, 1) 0 255
out size: (256, 1) 88 129
out size: (256, 1) 125 168


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
